In [1]:
import importlib
import ds
import augs
import labs

importlib.reload(ds)
importlib.reload(labs)
importlib.reload(augs)

from ds import *
from labs import *
from augs import *

In [2]:
import random
random.seed(1016) #707

# Swin Transfromer v2

In [3]:
# 모델 정보
model_name = 'microsoft/swinv2-base-patch4-window12-192-22k'

# DEF. Dataset and DataModule 

In [4]:
def prepare_example(image_path, processor, transform):
    # load image
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # load metas
    json_path = Path(image_path).with_suffix(".json")
    meta = load_json(json_path)

    augmented = transform(image=image)
    image = augmented['image']
    
    return processor(image, return_tensors="pt")

In [5]:
class D4Dataset(Dataset):
    def __init__(self, image_paths, targets, processor, transform=None):
        self.targets = targets
        self.processor = processor
        self.transform = transform
        self.image_paths = image_paths

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image_path = self.image_paths[idx]
        image = cv2.imread(image_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        json_path = Path(image_path).with_suffix(".json")
        meta = load_json(json_path)

        augmented = self.transform(image=image)
        image = augmented['image']
        encoding = self.processor(image, return_tensors="pt")

        target = int(self.targets[os.path.basename(image_path)])

        return {
            "pixel_values": encoding["pixel_values"].squeeze(0),
            "labels": torch.tensor(target, dtype=torch.long)
        }

In [6]:
import torch
torch.cuda.empty_cache()

In [7]:
class D4DataModule(LightningDataModule):
    def __init__(
        self,
        train_paths,
        valid_paths,
        trial_paths,
        target_dict,
        processor,
        batch_size=4,
        num_workers=6,
    ):
        super().__init__()
        self.train_paths = train_paths
        self.valid_paths = valid_paths
        self.trial_paths = trial_paths
        self.targets = target_dict
        self.processor = processor
        self.batch_size = batch_size
        self.num_workers = num_workers
        self.transforms = Transforms(target_size=192)

    def setup(self, stage=None):
        if stage == "fit":
            self.train_ds = D4Dataset(self.train_paths, 
                                      self.targets, 
                                      self.processor,
                                      self.transforms.make(50))
            self.valid_ds = D4Dataset(self.valid_paths, 
                                      self.targets, 
                                      self.processor,
                                      self.transforms.make(50))
        if stage == "test" or stage is None:
            self.trial_ds = D4Dataset(self.trial_paths, self.targets, self.processor, self.transforms.make(0))

    def train_dataloader(self):
        return DataLoader(
            self.train_ds,
            batch_size=self.batch_size,
            shuffle=True,
            num_workers=self.num_workers,
            collate_fn=default_data_collator
        )

    def val_dataloader(self):
        return DataLoader(
            self.valid_ds,
            batch_size=self.batch_size,
            shuffle=False,
            num_workers=self.num_workers,
            collate_fn=default_data_collator 
        )

    def test_dataloader(self):
        return DataLoader(
            self.trial_ds,
            batch_size=self.batch_size,
            shuffle=False,
            num_workers=self.num_workers,
            collate_fn=default_data_collator 
        )

# INIT. DM

In [8]:
image_paths = grep_files("/data/ephemeral/home/data/train", exts=['jpg'])
target_dict = load_csv_targets("/data/ephemeral/home/data/train.csv")
label_path = "/data/ephemeral/home/data/doc_classes.json"
label2id, id2label = make_doc_class_mapper(label_path)

0it [00:00, ?it/s]

In [9]:
from transformers import AutoImageProcessor, AutoModelForImageClassification
processor = AutoImageProcessor.from_pretrained(model_name)

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.48, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


In [10]:
train_images, valid_images, trial_images = split_ds(image_paths,  train_ratio=0.90,  valid_ratio=0.10, trial_ratio=0, seed=8315)

data_module = D4DataModule(
    train_paths=train_images,
    valid_paths=valid_images,
    trial_paths=trial_images,
    target_dict=target_dict,
    processor=processor,
    batch_size=16,
    num_workers=8
)

# DEF) Model

In [11]:
from transformers import Swinv2ForImageClassification

In [12]:
class CNN(pl.LightningModule):
    def __init__(self, label2id, id2label):
        super().__init__()
        num_classes = len(label2id)
        
        config = Swinv2Config.from_pretrained(model_name)
        config.num_labels = num_classes
        config.label2id = label2id
        config.id2label = id2label

        self.model = Swinv2ForImageClassification.from_pretrained(
            model_name,
            config=config,
            ignore_mismatched_sizes=True
        )
        self.model.train()
        self.model.config.label2id = label2id
        self.model.config.id2label = id2label

        metrics = {
            "accuracy": Accuracy(task="multiclass", num_classes=num_classes),
            # "top-3 accuracy" : MulticlassAccuracy(num_classes=10, top_k=3),
            "roc_auc": AUROC(task="multiclass", num_classes=num_classes),
            "precision": Precision(task="multiclass", num_classes=num_classes, average="macro"),
            "recall": Recall(task="multiclass", num_classes=num_classes, average="macro"),
            "F1": F1Score(task="multiclass", num_classes=num_classes, average="macro"),
        }

        self.train_metrics = MetricCollection(metrics, prefix="train_")
        self.valid_metrics = MetricCollection(metrics, prefix="valid_")

    def forward(self, pixel_values, labels=None):
        return self.model(pixel_values=pixel_values, labels=labels)

    def feed(self, batch):
        return self(batch["pixel_values"], batch["labels"])
        
    def training_step(self, batch, batch_idx):
        labels = batch["labels"]
        outputs = self.feed(batch)
    
        self.train_metrics.update(outputs.logits, labels)
        
        self.log("train_loss", outputs.loss)
        for name, metric in self.train_metrics.items():
            self.log(name, metric.compute(), prog_bar=True)

        # lr 기록
        optimizer = self.optimizers().optimizer
        current_lr = optimizer.param_groups[0]['lr']
        self.log('lr', current_lr)
        
        return outputs.loss

    def validation_step(self, batch, batch_idx):
        labels = batch["labels"]
        outputs = self.feed(batch)

        self.valid_metrics.update(outputs.logits, labels)
        
        self.log("valid_loss", outputs.loss)
        for name, metric in self.valid_metrics.items():
            self.log(name, metric.compute(), prog_bar=True)
        return outputs.loss

    def configure_optimizers(self):
        # lr 1e-5 -> 5e-6, scheduler 변경
        optimizer = torch.optim.AdamW(self.model.parameters(), lr=5e-6, weight_decay=1e-4)
        scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.2, patience=5)

        # 1. 지연 단계: total_iters 스텝 동안 초기 학습률 유지
        scheduler_delay = ConstantLR(optimizer, factor=1.0, total_iters=30)
        
        # 2. 웜업 단계: 그 다음 total_iters 스텝 동안 선형 웜업 수행
        scheduler_warmup = LinearLR(optimizer, start_factor=0.01, total_iters=10)
        
        # 3. 메인 스케줄링 단계 (1+2 스텝 이후): 코사인 어닐링 적용 (T_max: 전체 스텝에서 앞선 두 단계를 제외한 스텝 수)
        total_training_steps = 1000 # 총 학습 스텝 수
        scheduler_main = CosineAnnealingLR(optimizer, T_max=total_training_steps - 40, eta_min=1e-6)
        
        # scheduler_delay -> (30 스텝에서) -> scheduler_warmup -> (40 스텝에서) -> scheduler_main
        final_scheduler = SequentialLR(
            optimizer,
            schedulers=[scheduler_delay, scheduler_warmup, scheduler_main],
            milestones=[40, 50]
        )
        return {
            "optimizer": optimizer,
            "lr_scheduler": {
                "scheduler": final_scheduler,
                "interval": "step",
                "monitor": "valid_loss"
            }
        }
        
    def on_train_epoch_start(self):
        self.train_metrics.reset()

    def on_train_epoch_end(self):
        metrics = self.train_metrics.compute()
        for name, value in metrics.items():
            self.log(name, value)

    def on_validation_epoch_start(self):
        self.valid_metrics.reset()
    
    def on_validation_epoch_end(self):
        try:
            metrics = self.valid_metrics.compute()
            for k, v in metrics.items():
                self.log(k, v)
        except Exception as e:
            print(f"Metric compute error: {e}")
            
 

In [21]:
def make_proba_map_conv(image_paths, model, class_names=None):
    # 17개 클래스에 맞게 수정
    ids = list(range(17))  # [0, 1, 2, ..., 16]
    
    # 클래스 이름이 제공되지 않으면 기본값 사용
    if class_names is None:
        # 주의: 이 순서가 모델 학습 시 사용한 순서와 일치해야 합니다!
        class_names = [
            "계좌번호",
            "임신/출산 신청서", 
            "자동차 계기판",
            "입/퇴원 확인서",
            "진단서",
            "운전면허증",
            "진료/의료비 영수증",
            "외래/진료/통원/치료 확인서",
            "주민등록증",
            "여권",
            "(진료비/약제비) 납입 확인서",
            "약국/영수증",
            "처방전",
            "이력서",
            "소견서",
            "자동차 등록증",
            "자동차 번호판"
        ]
    
    labels = class_names
    
    model.eval()
    model = model.to('cuda') 
    rows = []

    # ✅ 추론 루프 - 확률 분포 계산
    for path in tqdm(image_paths):
        img = cv2.imread(path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        
        inputs = processor(img, return_tensors="pt")['pixel_values'].cuda()

        #inputs = {k: v.cuda() for k, v in inputs.items()}



        # 추론 - 확률 분포 계산
        with torch.no_grad():
            outputs = model(inputs) 
            logits = outputs.logits
            
            # 소프트맥스를 적용하여 확률 분포 계산
            probs = F.softmax(logits, dim=1)
            probs_np = probs.cpu().numpy().squeeze()  # (num_classes,)
            
            # 가장 높은 확률의 클래스 찾기
            pred_class = torch.argmax(logits, dim=1).item()
        
        # 확률을 소수점 2자리로 반올림
        probs_rounded = [float(f"{p:.2f}") for p in probs_np]
        
        # 예측 결과 생성
        guess = f"{labels[pred_class]} [{ids[pred_class]}]"
        
        # 결과 딕셔너리 생성
        item = {
            'code': os.path.basename(path),
            'guess': guess
        }
        
        # 각 클래스별 확률 추가
        for j in range(len(ids)):
            label = f"{labels[j]} [{ids[j]}]"
            item[label] = probs_rounded[j]
        
        rows.append(item)
    
    # 파일명 기준으로 정렬
    rows = sorted(rows, key=lambda x: x['code'])
    return rows

In [22]:
def save_results_to_csv(results, filename):
    """
    결과를 CSV 파일로 저장하는 함수
    
    Args:
        results: make_proba_map_layoutlmv3의 결과 (딕셔너리 리스트)
        filename: 저장할 파일명
    """
    import pandas as pd
    
    # 딕셔너리 리스트를 DataFrame으로 변환
    df = pd.DataFrame(results)
    
    # CSV로 저장
    df.to_csv(filename, index=False, encoding='utf-8-sig')  # 한글 지원을 위해 utf-8-sig 사용
    print(f"결과가 {filename}에 저장되었습니다.")
    
    return df

# Init Dashboard

In [14]:
exp_name = 'class-swin-base-129-transforms-aug'
wandb.init(project='docsy', name=exp_name)
wandb_logger = WandbLogger()

wandb: Currently logged in as: nodient91 (nodient91-kernel-academy) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


# RUN. Train

In [18]:
from pytorch_lightning.callbacks import EarlyStopping, ModelCheckpoint
from torchmetrics import MetricCollection, Accuracy, F1Score, Precision, Recall, AUROC
from torchmetrics.classification import MulticlassAccuracy
from torch.optim.lr_scheduler import ReduceLROnPlateau, ConstantLR, LinearLR, CosineAnnealingLR, CosineAnnealingWarmRestarts, SequentialLR

In [19]:
from transformers import Swinv2ForImageClassification, Swinv2Config

model_name = "microsoft/swinv2-base-patch4-window12-192-22k"

# 1. 정확한 config 로드
config = Swinv2Config.from_pretrained(model_name)
config.num_labels = len(label2id)
config.label2id = label2id
config.id2label = id2label

# 2. 모델 생성
model = Swinv2ForImageClassification.from_pretrained(
    model_name,
    config=config,
    ignore_mismatched_sizes=True  # label head 크기 안 맞는 문제 무시
)

Some weights of Swinv2ForImageClassification were not initialized from the model checkpoint at microsoft/swinv2-base-patch4-window12-192-22k and are newly initialized because the shapes did not match:
- classifier.weight: found shape torch.Size([21841, 1024]) in the checkpoint and torch.Size([17, 1024]) in the model instantiated
- classifier.bias: found shape torch.Size([21841]) in the checkpoint and torch.Size([17]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [20]:
early_stopping = EarlyStopping(monitor='valid_loss', patience=10, mode='min')
model_checkpoint = ModelCheckpoint(monitor="valid_loss", mode="min", save_top_k=2)

trainer = pl.Trainer(
    accelerator="gpu",
    precision="16-mixed",
    max_epochs=50,
    logger=wandb_logger,
    reload_dataloaders_every_n_epochs=1, 
    callbacks=[model_checkpoint]
    # callbacks=[model_checkpoint, early_stopping]
)

model = CNN(label2id, id2label)
trainer.fit(model, datamodule=data_module)

Using 16bit Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
Some weights of Swinv2ForImageClassification were not initialized from the model checkpoint at microsoft/swinv2-base-patch4-window12-192-22k and are newly initialized because the shapes did not match:
- classifier.weight: found shape torch.Size([21841, 1024]) in the checkpoint and torch.Size([17, 1024]) in the model instantiated
- classifier.bias: found shape torch.Size([21841]) in the checkpoint and torch.Size([17]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
You are using a CUDA device ('NVIDIA GeForce RTX 3090') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/gen

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/data/ephemeral/home/.pyenv/versions/py12/lib/python3.12/site-packages/torchmetrics/utilities/prints.py:43: UserWarning: No positive samples in targets, true positive value should be meaningless. Returning zero tensor in true positive score
  warnings.warn(*args, **kwargs)


Training: |          | 0/? [00:00<?, ?it/s]


Detected KeyboardInterrupt, attempting graceful shutdown ...


NameError: name 'exit' is not defined

In [19]:
trainer.save_checkpoint(f"./{exp_name}-last_epoch.ckpt")

In [20]:
wandb.finish()

epoch,▁▁▁▁▁▂▂▂▂▂▂▂▃▃▃▃▃▄▄▄▅▅▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇█
lr,▇▆▂▁▁▅█▆▅▁█▃▂▂▃▇▅▂▁▁▂▆█▇▂▂▁▃█▄▁▁▂██▂▂▃▇▇
train_F1,▁▂▂▃▄▄▅▄▅▄▅▄▄▅▆▅▄▆▇▆▆▆▇▆▇▇▇▇▇▅▇▇▇▇▄▅▇█▇▅
train_accuracy,▁▂▂▃▃▄▅▃▅▄▅▄▆▅▆▅█▄▆▅▆▇▇▅▇▇▅▅▇▆▆▇█▅▇▅▇▆▇█
train_loss,▆█▅▆▄▄▄▂▅▂▃▆▄▃▅▃▁▁▃▄▂▂▅▂▃▃▃▂▁▃▁▄▁▃▃▁▂▂▁▃
train_precision,▁▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇█████▇██████▇█▇▇▇██████
train_recall,▁▂▃▁▂▃▂▃▃▃▄▄▂▃▅▄▅▄▇▃▅▃▄▅▃▆▄▄▆▅▃▆▃▆█▄▆▄▇▆
train_roc_auc,▃▁▂▃▄▄▃▅▄▅▅▅▅▅▆▆▅▆▅▇▆▅▇▇▆▆▇▆▆█▆█▆██████▆
trainer/global_step,▁▁▁▁▂▃▃▃▃▃▃▃▃▃▃▃▃▃▄▄▄▄▅▅▅▆▆▆▆▆▆▆▆▆▇▇▇▇██
valid_F1,▁▃▃▃▆▆▃▅▅▄▅▆▅▅▇▇▆▇▇█▇▆█▇▇▇▆▇▇█▆█▇▆▆██▇█▇
valid_accuracy,▁▆▆▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇█▇█▇██▇█▇████▇█▇████


---

In [21]:
test_img_paths = grep_files("/data/ephemeral/home/data/test", exts=['jpg'])

0it [00:00, ?it/s]

In [22]:
import pandas as pd
import torch.nn.functional as F

def logits_to_prob_table(logits, class_names, labels=None):
    probs = F.softmax(logits, dim=1)  # (batch, num_classes)
    probs_np = probs.detach().cpu().numpy()
    batch_size, num_classes = probs_np.shape

    num_classes = probs_np.shape[1]
    if labels is not None:
        labels_np = labels.detach().cpu().numpy() if torch.is_tensor(labels) else labels
    else:
        labels_np = [''] * batch_size

    records = []
    columns = ['정답'] + [f'{name}' for name in class_names]
    for i in range(batch_size):
        row = [labels_np[i]] + ["{:.4f}".format(p) for p in probs_np[i]] 
        records.append(row)

    df = pd.DataFrame(records, columns=columns)
    return df

In [24]:
model_path = "/data/ephemeral/home/doc_classification-main/chy/cnn/exp-swin-base-129-transforms-aug-last_epoch.ckpt"
model = CNN.load_from_checkpoint(model_path, id2label=id2label, label2id=label2id)

Some weights of Swinv2ForImageClassification were not initialized from the model checkpoint at microsoft/swinv2-base-patch4-window12-192-22k and are newly initialized because the shapes did not match:
- classifier.weight: found shape torch.Size([21841, 1024]) in the checkpoint and torch.Size([17, 1024]) in the model instantiated
- classifier.bias: found shape torch.Size([21841]) in the checkpoint and torch.Size([17]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [25]:
import torch.nn.functional as F

# 1. 추론 실행
results = make_proba_map_conv(
    image_paths, 
    model
)

# 2. CSV 저장
df = save_results_to_csv(results, "[AJH]swin-proba-map-desque-test.csv")

# 3. 결과 확인
print(f"총 {len(df)}개 이미지 처리 완료")
print(df.head())

  0%|          | 0/1570 [00:00<?, ?it/s]

결과가 [AJH]swin-proba-map-desque-test.csv에 저장되었습니다.
총 1570개 이미지 처리 완료
                   code                  guess  계좌번호 [0]  임신/출산 신청서 [1]  \
0  002f99746285dfdd.jpg           자동차 번호판 [16]       0.0            0.0   
1  008ccd231e1fea5d.jpg  (진료비/약제비) 납입 확인서 [10]       0.0            0.0   
2  008f5911bfda7695.jpg  (진료비/약제비) 납입 확인서 [10]       0.0            0.0   
3  009235e4c9c07af5.jpg                진단서 [4]       0.0            0.0   
4  00b2f44967580c74.jpg           자동차 번호판 [16]       0.0            0.0   

   자동차 계기판 [2]  입/퇴원 확인서 [3]  진단서 [4]  운전면허증 [5]  진료/의료비 영수증 [6]  \
0          0.0           0.0      0.0        0.0             0.0   
1          0.0           0.0      0.0        0.0             0.0   
2          0.0           0.0      0.0        0.0             0.0   
3          0.0           0.0      1.0        0.0             0.0   
4          0.0           0.0      0.0        0.0             0.0   

   외래/진료/통원/치료 확인서 [7]  주민등록증 [8]  여권 [9]  (진료비/약제비) 납입 확인서 [10]  약국/영수증

In [24]:
model.eval()
model = model.to('cuda') 

In [25]:
preds = []
for p in tqdm(test_img_paths):
    img = cv2.imread(p)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = processor(img, return_tensors="pt")['pixel_values']
    img = img.cuda()
    outs = model(img)

    predict = torch.argmax(outs.logits, dim=1)
    pred = (p, predict.item())
    
    # top3_values, top3_indices = torch.topk(outs.logits, k=3, dim=1)
    # pred = (top3_values.cpu().numpy(), top3_indices.cpu().numpy())
    preds.append(pred)

  0%|          | 0/3142 [00:00<?, ?it/s]

In [26]:
def write_csv_value(csv_path, preds):
    df = pd.read_csv(csv_path)
    for i, item in enumerate(preds):
        filename = os.path.basename(item[0])
        df.loc[df['ID'] == filename, 'target'] = item[1]
        
    df.to_csv(csv_path, index=False, encoding='utf-8')

In [27]:
csv_path = "/data/ephemeral/home/pred3.csv"
write_csv_value(csv_path, preds)

In [38]:
# 정의된 클래스 정보
ids = [1, 3, 4, 6, 7, 10, 11, 12, 13, 14]
labels = ['임신/출산', '입퇴원확인서', '진단서', '의료비영수증', '진료통원확인서', '납입확인서', '약국영수증', '처방전', '이력서', '소견서']

# model, processor는 이미 학습된 상태라고 가정
rows = make_proba_map(test_img_paths, model=model, processor=processor, labels=labels, ids=ids)

Predicting:   0%|          | 3/3142 [00:00<02:15, 23.11it/s]


IndexError: list index out of range

# 체크포인트 로드

In [ ]:
ckpt_path = "/data/ephemeral/home/doc_classification-main/chy/cnn/exp-swin-large-192-transforms-aug-last_epoch.ckpt"
model = CNN.load_from_checkpoint(ckpt_path, id2label=id2label, label2id=label2id)

In [29]:
model.eval()
model = model.to('cuda') 

In [30]:
img = cv2.imread(test_img_paths[0])
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
img = processor(img, return_tensors="pt")['pixel_values']
img = img.cuda()
outs = model(img)

In [31]:
label_trans = {
    "account_number":"계좌번호",
    "application_for_payment_of_pregnancy_medical_expenses": "임신/출산 신청서",
    "car_dashboard": "자동차 계기판",
    "confirmation_of_admission_and_discharge": "입퇴원 확인서",
    "diagnosis": "진단서",
    "driver_lisence": "운전면허증",
    "medical_bill_receipts": "진료/의료비 영수증",
    "medical_outpatient_certificate": "(외래)진료(통원/치료) 확인서",
    "national_id_card": "주민등록증",
    "passport": "여권",
    "payment_confirmation": "(진료비/약제비) 납입 확인서",
    "pharmaceutical_receipt": "약국/영수증",
    "prescription": "처방전",
    "resume": "이력서",
    "statement_of_opinion": "소견서",
    "vehicle_registration_certificate": "자동차 등록증",
    "vehicle_registration_plate": "자동차"
}

In [34]:
make_proba_map(image_paths)

NameError: name 'get_words' is not defined

# 체크포인트 저장하고 로드하고 로짓으로 TOP3 혹은 전체 표 작성

In [32]:
class_names = [label_trans[k] for k in label2id.keys()]
logits_to_prob_table(outs.logits, class_names=class_names)

,정답,계좌번호,임신/출산 신청서,자동차 계기판,입퇴원 확인서,진단서,운전면허증,진료/의료비 영수증,(외래)진료(통원/치료) 확인서,주민등록증,여권,(진료비/약제비) 납입 확인서,약국/영수증,처방전,이력서,소견서,자동차 등록증,자동차
0,,0.0000,0.0000,0.0000,0.6340,0.0000,0.0000,0.0000,0.3659,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0001,0.0000,0.0000


In [ ]:
show_img(test_img_paths[1])